In [1]:
import random

In [2]:

class AGInt:
    def __init__(self,procesos:list, num_procesadores: int, underload: int, overload: int, sizew:int,  
                 rango_valores:list, num_cromosomas:int, num_generaciones=200, prob_cruce=0.8, prob_mutacion=0.05, porc_elitismo=0.1):
        # Algoritmo genetico
        self.rv = rango_valores
        self.tam_poblacion = num_cromosomas
        self.num_generaciones = num_generaciones
        self.prob_cruce = prob_cruce
        self.prob_mutacion = prob_mutacion
        self.porc_elitismo = porc_elitismo

        # Distribución de cargas
        self.num_procesadores = num_procesadores
        self.underload = underload
        self.overload = overload
        self.procesos = procesos
        self.sizew = sizew
        self.solucion = []
        self.distribucion = [0] * num_procesadores
        self.carga_maxima = None

    
    def crear_cromosoma(self):
        cromosoma = self.rv.copy()
        random.shuffle(cromosoma)
        return cromosoma
    
    def crear_poblacion(self):
        return [self.crear_cromosoma() for _ in range(self.tam_poblacion)]

    def seleccion(self, poblacion:list):
        cromosoma = random.choice(poblacion)
        return cromosoma

    def crossover(self, p1: list, p2: list):
        L = len(p1)
        H1 = [None] * L
        H2 = [None] * L

        l1 = random.randint(0, L - 1)
        l2 = random.randint(l1 + 1, L)

        # Llenamos los hijos con los valores de los intervalos definidos del padre contrario
        H1[l1:l2] = p2[l1:l2]
        H2[l1:l2] = p1[l1:l2]
        # print(f"{H1}\n{H2}\n")

        for h in range(2):
            if h == 0:
                Ha = H1
                Pa = p1
                Pc = p2
            else:
                Ha = H2
                Pa = p2
                Pc = p1

            # Completamos con los valores del mismo padre que no se encuentren en el hijo
            for i in range(len(Ha)):
                if (Ha[i] == None):
                    if (Pa[i] not in Ha):
                        Ha[i] = Pa[i]
            
            # Terminamos completando al hijo con los valores del padre contrario que no se encuentren
            #  en el orden de izquierda a derecha
            for i in range(len(Pc)):
                if (Pc[i] not in Ha):
                    Ha[Ha.index(None)] = Pc[i]
            
            if h == 0:
                H1 = Ha
            else:
                H2 = Ha

        # print(f"{H1}\n{H2}")
        # print(len(set(H1)) == len(H1) and len(set(H2)) == len(H2)) # Comprobar que no hay elementos repetidos en ambas listas, todo se hizo correctamente
        
        return H1, H2

    def distribuir(self, cromosoma:list, ventana:list):
        carga_procesador = self.distribucion.copy()
        i = 0
        for asignar in cromosoma:
            if i < self.num_procesadores:
                carga_procesador[i] += ventana[asignar]
                i += 1
            else:
                i = 0
                carga_procesador[i] += ventana[asignar]
                i += 1

        return carga_procesador

    def entrenar(self):
        for i in range(0,len(self.procesos), self.sizew):
            ven = self.procesos[i:i+self.sizew]

            if len(ven) < self.sizew:
                ceros = self.sizew - len(ven)
                ven += [0] * ceros
            
            poblacion = self.crear_poblacion()
            for _ in range(self.num_generaciones):
                poblacion = self.nueva_poblacion(poblacion, ven)
            
            
            best = self.best(poblacion, ven)
            self.distribucion = self.distribuir(best, ven)
            
            self.solucion.append( {"ventana":ven, "cromosoma":best, "fitness":self.fitness(best, ven), "distribucion":self.distribucion} )

    def nueva_poblacion(self, poblacion:list, ventana:list):
        nueva_poblacion = []
        nueva_poblacion.extend(self.elite(poblacion, ventana))

        while len(nueva_poblacion) < self.tam_poblacion:

            if random.random() < self.prob_cruce: 
                padre1 = self.seleccion(poblacion)
                padre2 = self.seleccion(poblacion)
                
                hijo1, hijo2 = self.crossover(padre1, padre2)

                nueva_poblacion.extend([hijo1, hijo2])

        return nueva_poblacion
    
    def elite(self, poblacion: list, ventana:list):
        totalE = int(self.tam_poblacion * self.porc_elitismo)
        return sorted(poblacion, key=lambda k: self.fitness(k,ventana), reverse=True)[:totalE]
    
    def fitness(self, cromosoma:list, ventana:list):
        carga_procesador = self.distribuir(cromosoma, ventana)

        self.carga_maxima = max(carga_procesador)
        cargasAcep = sum(1 for carga in carga_procesador if self.underload < carga < self.overload)
        promCarga = sum(carga / self.carga_maxima for carga in carga_procesador) / self.num_procesadores

        return (1.0 / self.carga_maxima) * promCarga * (cargasAcep / self.num_procesadores)
    
    def best(self, poblacion, ventana):
        return max(poblacion, key=lambda k: self.fitness(k, ventana))

Parametros

In [3]:
n_procesos = 3600
n_procesadores = 4
overloaded = (n_procesos // n_procesadores) + 100
sizew = 6
rv = [i for i in range(sizew)]

Implementación

In [ ]:
# La suma de esta lista es de 3600 y es estatica por motivos de prueba, pero los elementos del arreglo deben generarse de forma aleatorio entre 5 y 50 y la de sus elementos debe ser igual a n_procesos
procesos = [304,542,139,116,51,44,155,172,175,202,35,191,232,277,42,153,176,76,209,129,180]

divCarAG = AGInt(
    procesos = procesos,
    num_procesadores = n_procesadores, 
    sizew=sizew,
    underload = 50, overload = overloaded, 
    num_cromosomas = 100, rango_valores=rv)

divCarAG.entrenar()
sol = divCarAG.solucion

print("Carga de procesos:", procesos)
print("\n- La mejor division de cargas es:")
for i, slide in enumerate(sol, start=1):
    print(f"Carga {i}: {slide['ventana']}")
    print(f"Cromosoma: {slide['cromosoma']}")
    print(f"Fitness: {slide['fitness']}")

    print(f"- Distribución de los procesadores:")
    for j, proceso in enumerate(slide["distribucion"], start=0):
        print(f"Procesador {j}: {proceso}") 

    print("\n")

print("- Tiempo total: ", max(sol[-1]["distribucion"]))